---
# 課題5　                                                                                                 

### 男性 (学生) 提案と女性 (大学) 提案を入れ替えてマッチング結果を比べる

#### 3年　高橋　雅士　　　　コードはこちら(https://github.com/masashitshit/MyMatching.jl/blob/master/src/MyMatching.jl)
---

今までのコードに、提案側と応答側を入れ替えてマッチングするコードを付け加えました。（今まではOne=>ManyだったがMany=>Oneでマッチングさせることもできるように。）

新しいmatchingはreverse = true/false で提案する側を入れ替えます。

提案側と応答側を入れ替えたとき、マッチングの質に変化があるかを調べます。
まずはOne-to-oneについて。

In [1]:
using MyMatching

In [2]:
using Matching
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [3]:
m = 10 
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
reverse = false
matching(prop_prefs,resp_prefs,reverse)   #男性　→　女性のとき

([5,0,0,10,2,4,6,8,9,0],[0,5,0,6,1,7,0,8,9,4])

In [4]:
reverse = true
matching(prop_prefs,resp_prefs,reverse)   #女性　→　男性のとき

([5,0,0,10,2,4,6,8,9,0],[0,5,0,6,1,7,0,8,9,4])

In [5]:
m = 10         #女性を減らしてみる
n = 4
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
reverse = false
matching(prop_prefs,resp_prefs,reverse)   #男性　→　女性のとき

([0,0,0,2,1,4,0,3,0,0],[5,4,8,6])

In [6]:
reverse = true
matching(prop_prefs,resp_prefs,reverse)   #女性　→　男性のとき

([0,0,0,2,1,4,0,3,0,0],[5,4,8,6])

提案する側を入れ替えてもマッチング結果は安定的に同じでした。次はMany-to-manyについて。

(capsは各大学にk個の受け入れ枠を与えます。)

In [7]:
m = 30
n = 6
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき

([3,2,0,0,5,0,0,5,2,0  …  4,5,3,0,0,0,0,1,0,0],[28,18,0,2,13,9,23,15,1,21,0,0,5,22,8,11,0,0],[1,4,7,10,13,16,19])

In [8]:
reverse = true
matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき

([3,2,0,0,5,0,0,5,2,0  …  4,5,3,0,0,0,0,1,0,0],[18,28,0,13,9,2,23,1,15,21,0,0,8,22,5,11,0,0],[1,4,7,10,13,16,19])

In [9]:
m = 50
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
result1 = matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき

([7,0,0,7,2,0,0,0,1,0  …  1,10,10,8,8,3,4,0,0,0],[16,9,41,38,5,20,46,12,25,47  …  37,44,45,26,15,24,0,42,17,43],[1,4,7,10,13,16,19,22,25,28,31])

In [10]:
reverse = true
result2 = matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき

([7,0,0,7,2,0,0,0,1,0  …  1,10,10,0,8,3,4,0,8,0],[41,9,30,5,20,38,25,46,12,13  …  1,26,45,49,24,0,15,43,42,17],[1,4,7,10,13,16,19,22,25,28,31])

In [11]:
result1[1] -result2[1]　　#提案を入れ替えたときどこに違いがあるか

50-element Array{Int64,1}:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  ⋮
  0
  0
  0
  0
  0
  8
  0
  0
  0
  0
 -8
  0

In [12]:
a = [prop_prefs,resp_prefs,count(i -> (i != 0),result1[1] -result2[1]),find(result1[1]-result2[1].>0)]　
#[propの選好,respの選好,propのマッチで変化した数,変化したpropのインデックス]

4-element Array{Any,1}:
  Array{Int64,1}[[7,2,1,4,5,8,10,6,3,9,0],[6,4,5,1,3,0],[8,7,5,2,4,9,10,1,6,3,0],[7,10,1,2,4,3,5,6,9,0],[8,6,5,7,4,2,0],[1,10,2,4,7,6,8,0],[7,0],[5,0],[3,8,1,6,7,4,0],[9,1,7,6,0]  …  [1,7,5,8,0],[9,2,5,10,7,4,1,0],[8,6,1,10,2,0],[7,9,3,10,5,4,2,8,1,6,0],[6,1,8,3,5,7,9,0],[1,6,4,10,3,5,8,7,2,9,0],[1,3,4,8,9,7,2,10,6,5,0],[2,6,0],[4,7,8,10,0],[10,6,4,2,8,3,5,7,0]]                                                                                                                                                                                                         
  Array{Int64,1}[[37,11,16,13,50,41,9,23,30,15  …  39,31,40,33,29,6,5,46,35,0],[5,1,18,38,37,20,39,22,28,45  …  14,33,21,13,44,34,16,15,11,0],[25,46,12,31,32,28,15,10,39,34  …  50,35,26,27,2,47,19,29,4,0],[48,17,32,31,24,15,13,27,20,21  …  8,34,37,16,40,39,25,5,28,0],[48,27,13,22,23,35,3,8,6,31  …  39,50,40,37,9,36,28,7,30,0],[47,31,7,27,19,49,0],[48,37,21,1,4,3,6,32,12,8  …  40,23,17,31,41,34,39,11,18,0],[

In [13]:
m = 50
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
result1 = matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき
reverse = true
result2 = matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき
b =[prop_prefs,resp_prefs,count(i -> (i != 0),result1[1] -result2[1]),find(result1[1]-result2[1].>0)]

4-element Array{Any,1}:
  Array{Int64,1}[[9,3,0],[2,4,3,9,1,8,5,7,0],[4,6,5,0],[10,8,1,2,7,5,4,3,0],[3,7,1,4,6,2,9,8,5,0],[8,1,2,6,4,9,7,5,3,0],[6,0],[2,5,9,7,1,10,0],[7,5,9,4,3,0],[7,9,2,0]  …  [2,6,7,8,9,5,4,3,0],[2,10,9,7,8,1,6,3,0],[5,8,2,0],[9,0],[4,5,0],[8,1,9,7,2,6,10,4,3,5,0],[9,0],[10,5,7,8,1,6,4,0],[7,1,6,2,10,8,0],[9,7,4,5,3,10,6,2,0]]                                                                                                                                            
  Array{Int64,1}[[46,35,16,29,28,36,17,47,3,1,4,6,32,0],[20,3,36,8,43,6,32,11,24,0],[49,1,15,22,44,25,26,17,21,11  …  6,32,28,10,36,19,18,45,42,0],[1,16,4,34,6,11,5,9,21,18  …  8,50,43,26,17,24,12,25,23,0],[8,20,29,7,48,5,42,16,37,25,36,4,27,0],[27,1,13,2,42,26,0],[2,42,13,19,31,46,14,49,10,32  …  16,23,12,28,43,3,37,7,17,0],[43,27,48,8,42,19,30,6,33,24,49,21,35,40,46,47,7,25,34,0],[34,28,18,22,0],[25,23,26,14,33,12,10,18,34,16  …  19,22,4,9,32,37,35,43,30,0]]
 0                                            

In [14]:
m = 50
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
result1 = matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき
reverse = true
result2 = matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき
c =[prop_prefs,resp_prefs,count(i -> (i != 0),result1[1] -result2[1]),find(result1[1]-result2[1].>0)]

4-element Array{Any,1}:
  Array{Int64,1}[[9,10,4,5,1,0],[2,6,0],[2,7,8,3,4,1,5,10,6,0],[2,3,9,7,8,4,6,0],[8,2,6,7,9,0],[1,6,3,10,7,0],[1,7,10,2,4,5,0],[5,9,10,2,3,6,7,0],[5,7,3,2,6,1,8,0],[4,5,1,2,10,8,9,3,0]  …  [1,2,5,3,8,7,0],[4,3,5,1,7,8,2,10,0],[1,9,6,8,0],[4,0],[9,8,5,7,3,10,0],[9,2,0],[10,3,4,5,6,9,2,1,7,0],[10,1,0],[8,1,6,2,5,4,10,3,7,9,0],[5,7,0]]                                                                                                                                                                                               
  Array{Int64,1}[[15,30,6,31,8,46,28,12,41,39,24,36,16,10,34,0],[8,30,43,48,19,44,33,32,39,27,9,0],[27,40,45,23,16,1,6,26,30,48  …  37,38,47,35,49,11,44,50,29,0],[27,47,42,3,21,24,44,11,14,19,7,28,1,31,48,6,0],[12,15,46,33,2,45,28,41,37,38,4,10,26,36,6,24,40,11,0],[39,42,49,47,28,41,18,4,0],[12,9,45,37,20,15,13,10,31,2  …  41,26,32,29,11,23,33,3,8,0],[22,46,12,25,38,24,41,50,37,21  …  30,4,23,15,17,10,26,1,28,0],[14,10,7,8,35,21,20,44,34,28,15,43

In [15]:
m = 50
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
result1 = matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき
reverse = true
result2 = matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき
d =[prop_prefs,resp_prefs,count(i -> (i != 0),result1[1] -result2[1]),find(result1[1]-result2[1].>0)]

4-element Array{Any,1}:
  Array{Int64,1}[[8,0],[8,6,9,10,4,7,1,2,3,5,0],[7,10,0],[8,9,6,5,7,10,1,2,3,4,0],[1,9,3,0],[10,6,9,8,7,0],[6,10,9,0],[9,1,10,8,0],[5,1,6,8,3,4,7,10,2,0],[2,4,0]  …  [1,6,7,5,10,2,3,0],[6,9,2,1,3,0],[3,5,10,7,9,4,6,0],[7,10,5,0],[1,4,3,10,2,9,7,5,0],[4,5,7,3,1,2,8,0],[5,0],[6,7,1,3,4,2,8,5,10,9,0],[10,0],[3,6,7,2,5,9,1,0]]                                                           
  Array{Int64,1}[[40,15,37,13,45,18,30,24,42,41  …  2,39,23,17,9,49,19,35,4,0],[2,20,40,38,0],[36,1,18,49,34,38,35,8,3,37,20,0],[22,20,6,23,2,0],[30,0],[45,35,4,22,14,13,2,38,24,30  …  17,9,6,1,12,37,29,19,7,0],[4,40,41,17,34,47,18,50,10,3  …  37,31,11,49,13,42,26,35,14,0],[4,29,7,23,21,2,40,26,0],[21,44,8,45,39,32,27,43,1,19  …  2,36,18,24,9,29,28,7,41,0],[23,44,46,3,50,48,40,0]]
 0                                                                                                                                                                                                              

In [16]:
m = 50
n = 10
prop_prefs, resp_prefs = mat2vecs.(random_prefs(m, n)) 
k = 3
caps = k*ones(Int64,n)
reverse = false
result1 = matching(prop_prefs,resp_prefs,caps,reverse)   #学生　→　大学のとき
reverse = true
result2 = matching(prop_prefs,resp_prefs,caps,reverse)   #大学　→　学生のとき
e =[prop_prefs,resp_prefs,count(i -> (i != 0),result1[1] -result2[1]),find(result1[1]-result2[1].>0),result1[1]]

5-element Array{Any,1}:
  Array{Int64,1}[[10,1,9,6,4,3,8,7,5,0],[6,4,2,3,9,1,5,7,0],[3,6,5,1,4,0],[6,7,1,8,3,4,5,9,10,2,0],[10,5,7,4,9,1,2,8,6,3,0],[7,10,9,8,3,1,5,6,2,4,0],[7,10,5,2,9,0],[10,8,7,9,0],[10,9,2,5,1,3,0],[9,2,1,7,4,3,0]  …  [2,9,4,0],[5,0],[10,4,1,6,8,9,5,7,2,0],[9,2,7,10,5,8,3,1,6,0],[6,9,10,1,2,8,7,3,0],[5,6,8,10,2,1,9,4,7,3,0],[10,0],[10,7,6,4,1,8,0],[8,0],[2,5,7,0]]                                                                                                                                               
  Array{Int64,1}[[16,49,15,37,10,35,27,11,4,1,20,28,19,39,0],[15,5,41,20,48,22,29,19,11,30  …  33,38,9,31,45,34,17,18,32,0],[16,7,42,22,23,26,14,20,47,3  …  39,15,8,33,49,13,28,6,29,0],[19,5,29,40,47,41,25,27,20,28,45,37,21,0],[4,11,2,5,8,1,36,21,20,24  …  18,10,17,33,47,12,28,40,44,0],[16,9,39,1,15,43,3,27,44,17  …  34,28,18,49,29,2,42,8,23,0],[21,12,46,39,7,14,19,23,25,10  …  30,43,49,50,36,37,9,22,42,0],[14,1,36,24,16,27,32,15,5,35  …  6,8,48,30,42,29,47,21,12,0]

Many-to-manyでは提案を入れ替えるとマッチング結果が少しだけ変わったり、変わらなかったりします。どうして一見安定そうな結果にわずかな違いが生じるのかについて調べてみます。

上のa,b,c,d,eの5つは選好以外は条件が同じなので理由として選好の違いが考えられます。各選好についての特徴を見てみます。

In [17]:
[[sum(length(a[1][i]) for i in 1:m),sum(length(b[1][i]) for i in 1:m),sum(length(c[1][i]) for i in 1:m),
    sum(length(d[1][i]) for i in 1:m),sum(length(e[1][i]) for i in 1:m)],[a[3],b[3],c[3],d[3],e[3]]]       #[propの選好の数]

2-element Array{Array{Int64,1},1}:
 [345,327,342,313,356]
 [4,0,7,0,3]          

In [18]:
[[sum(length(a[2][i]) for i in 1:n),sum(length(b[2][i]) for i in 1:n),sum(length(c[2][i]) for i in 1:n),
    sum(length(d[2][i]) for i in 1:n),sum(length(e[2][i]) for i in 1:n)],[a[3],b[3],c[3],d[3],e[3]]]       #[respの選好の数]

2-element Array{Array{Int64,1},1}:
 [258,231,214,199,255]
 [4,0,7,0,3]          

各主体が対象とする相手の数が多ければ多いほどマッチングが変化する余地が多そうな気がしましたが、
prop,respの選好の数が多い（=各主体がより多くの相手とマッチングできる可能性をもつ）ことはあまりマッチング結果と関係がなさそうです。

In [19]:
prop_popa = [[count(p-> i<=p<i+1,a[1][j]) for i in 1:n] for j in 1:m]        #resp[i]がpropの選好の中に何回登場するか
prop_popb = [[count(p-> i<=p<i+1,b[1][j]) for i in 1:n] for j in 1:m]        
prop_popc = [[count(p-> i<=p<i+1,c[1][j]) for i in 1:n] for j in 1:m]        
prop_popd = [[count(p-> i<=p<i+1,d[1][j]) for i in 1:n] for j in 1:m]        
prop_pope = [[count(p-> i<=p<i+1,e[1][j]) for i in 1:n] for j in 1:m]        

resp_popa = [[count(p-> i<=p<i+1,a[2][j]) for i in 1:m] for j in 1:n]       #prop[i]がrespの選好の中に何回登場するか
resp_popb = [[count(p-> i<=p<i+1,b[2][j]) for i in 1:m] for j in 1:n]  
resp_popc = [[count(p-> i<=p<i+1,c[2][j]) for i in 1:m] for j in 1:n]  
resp_popd = [[count(p-> i<=p<i+1,d[2][j]) for i in 1:m] for j in 1:n]  
resp_pope = [[count(p-> i<=p<i+1,e[2][j]) for i in 1:m] for j in 1:n]

10-element Array{Array{Int64,1},1}:
 [1,0,0,1,0,0,0,0,0,1  …  0,0,0,0,0,0,0,0,1,0]
 [1,1,1,0,1,0,0,0,1,0  …  1,0,1,0,1,0,1,1,1,1]
 [1,0,1,0,0,1,1,1,0,0  …  1,1,0,0,0,0,1,1,1,1]
 [0,0,0,0,1,0,0,0,0,0  …  1,0,0,0,1,0,1,0,0,0]
 [1,1,0,1,1,0,0,1,0,1  …  0,0,0,1,0,1,1,1,0,0]
 [1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,1,0,1,0,1,1]
 [0,0,0,0,1,1,1,1,1,1  …  0,1,1,0,0,1,0,0,1,1]
 [1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,0,1,1,1,1,1]
 [0,0,0,0,0,1,0,0,0,0  …  0,0,0,0,0,0,0,0,0,0]
 [0,0,0,0,0,0,0,0,0,0  …  0,0,0,0,0,0,1,1,0,0]

In [20]:
prop_numa = [sum(prop_popa[i])for i in 1:m]                #propの選好に1 ~ nまでが何回ふくまれていたか
prop_numb = [sum(prop_popb[i])for i in 1:m]    
prop_numc = [sum(prop_popc[i])for i in 1:m]    
prop_numd = [sum(prop_popd[i])for i in 1:m]    
prop_nume = [sum(prop_pope[i])for i in 1:m]

resp_numa = [sum(resp_popa[i])for i in 1:n] 
resp_numb = [sum(resp_popb[i])for i in 1:n] 
resp_numc = [sum(resp_popc[i])for i in 1:n] 
resp_numd = [sum(resp_popd[i])for i in 1:n] 
resp_nume = [sum(resp_pope[i])for i in 1:n] 

10-element Array{Int64,1}:
 14
 32
 33
 13
 28
 44
 23
 49
  2
  7

In [21]:
[[sum(prop_numa)/n,sum(prop_numb)/n,sum(prop_numc)/n,sum(prop_numd)/n,sum(prop_nume)/n],
    [sum(resp_numa)/m,sum(resp_numb)/m,sum(resp_numc)/m,sum(resp_numd)/m,sum(resp_nume)/m]]   #各選好の出方の平均

2-element Array{Array{Float64,1},1}:
 [29.5,27.7,29.2,26.3,30.6]
 [4.96,4.42,4.08,3.78,4.9] 

In [22]:
[[var(prop_numa),var(prop_numb),var(prop_numc),var(prop_numd),var(prop_nume)],
    [var(resp_numa),var(resp_numb),var(resp_numc),var(resp_numd),var(resp_nume)]]   #各選好にどの程度出方のばらつきがあるか

2-element Array{Array{Float64,1},1}:
 [7.47959,7.84531,9.85143,8.8902,8.72]    
 [89.2889,253.433,126.933,303.656,242.056]

In [23]:
[a[3],b[3],c[3],d[3],e[3]]

5-element Array{Int64,1}:
 4
 0
 7
 0
 3

平均的にどれだけ相手グループの選好に入るかや、どれだけ選好の入り方にばらつきがあるかといったこともマッチング結果に関係がなさそうです。

最後に実際にマッチング結果が変化したケースについて、変化したprop(resp)の相手と志望順位との関係について調べてみます。

In [24]:
change = [[result1[1][e[4][i]] for i in 1:length(e[4])],[result2[1][e[4][i]] for i in 1:length(e[4])]]   
#eにおいてのマッチングの変化

2-element Array{Array{Int64,1},1}:
 [4]
 [0]

このケースにおいてpropにとってより良い結果になったかどうか調べます。

In [25]:
[findfirst(e[1][e[4][1]],change[1][1]),findfirst(e[1][e[4][1]],change[2][1])]       #propのマッチ相手の志望順位の変化

2-element Array{Int64,1}:
  5
 10

変化したpropのマッチング相手はpropにとってより志望順位の低い相手となってしまいました。
最後に各マッチング相手の志望順位の和をprop,respそれぞれ出してみます。（小さいほど平均的により志望度の高い相手とマッチングしているといえる）

In [26]:
prop_ranking = [findfirst(prop_prefs[i],result1[1][i]) for i in 1:m]
#学生　→　大学のとき　学生
resp_ranking = [[findfirst(resp_prefs[i],result1[2][j]) for j in result1[3][i]:result1[3][i+1]-1] for i in 1:n]
#学生　→　大学のとき　大学
prop_ranking_r = [findfirst(prop_prefs[i],result2[1][i]) for i in 1:m]
#大学　→　学生のとき　学生
resp_ranking_r = [[findfirst(resp_prefs[i],result2[2][j]) for j in result2[3][i]:result2[3][i+1]-1] for i in 1:n]
#大学　→　学生のとき　大学
[sum(prop_ranking),sum(prop_ranking_r),sum(prop_ranking)-sum(prop_ranking_r),
        sum([sum(resp_ranking[i])for i in 1:n]),sum([sum(resp_ranking_r[i])for i in 1:n]),
        sum([sum(resp_ranking[i])for i in 1:n])-sum([sum(resp_ranking_r[i])for i in 1:n])]

6-element Array{Int64,1}:
 216
 218
  -2
 130
 144
 -14

prop、resp両社にとってマッチング相手の平均的な志望度が下がってしまいました。提案入れ替え後のマッチング結果は、提案入れ替え前のマッチング結果に対してパレート劣位ということになります。（eのケースにおいて。他のケースだとpropとrespの結果にトレードオフがあったりもする。）

---
全体を通してマッチング結果がなぜ、どのように変わるかを調べてみましたが解明できませんでした。
ただマッチング結果は想像よりも入れ替え前後で共通した部分が多く、時には完全に一致していることが見て取れます。
そこにノイズ（と見て良いのかも不明）が加わる理由はよくわかりませんでした。


マッチング結果はprop、respの選好の具合に強く影響されるので、選好を固定することなく調べてみるのは良くない方針でした。

---